Sample of model preparation that are used in creating ensemble classifier. The parameters of lightGBM and weight of bert/roberta will be different for different datasets or tasks. Here we have provided a sample code. For the random forest, we used  model of Arya et al. 

#Installation of necessary libabries

In [ ]:
!pip install datasets
!pip install transformers

In [ ]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] ='0' #gpu

In [1]:
datapath='Dataset.csv' #enter the correct path

In [ ]:
from transformers import AutoTokenizer
from torch import nn
from transformers import Trainer
import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import numpy as np
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset
import datasets
from sklearn import preprocessing
from sklearn.model_selection import KFold,StratifiedKFold

#Tokenizer & Model

In [ ]:
ce) or not
tokenizer = AutoTokenizer.from_pretrained("roberta-base") #for bert, change to bert-base-uncased


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess_function(examples, text_column_name = "text"):
    return tokenizer(examples[text_column_name], truncation=True)

ective. 
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=12) #for bert, change to bert-base-uncased and change num_labels for different dataset


In [ ]:
import pandas as pd
issue_df = pd.read_csv(datapath)
issue_df = issue_df[['Text Content','Code']] #will only  consider text and label for this problem
issue_df = issue_df.rename(columns={'Text Content': 'text', 'Code': 'label'}) #rename the columns
print(issue_df.info())

label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(issue_df['label'])

issue_df['label'] = label_encoder.transform(issue_df['label'])

# Training and Testing of Bert/Roberta
We have used cross fold validation where the dataset has been split into 10-folds using a stratified sampling. Each fold contains the data of each class in the same proportion and no two folds has any overlap between themselves


In [ ]:
#cross-fold validation
skf = StratifiedKFold(n_splits=10, random_state=10, shuffle=True)

fold=0
for trainval_index, test_index in skf.split(issue_df,issue_df['label']):  
    fold=fold+1

    trainval_df = issue_df.iloc[trainval_index]
    test_df = issue_df.iloc[test_index]

    #split for getting validation dataset
    train_df, val_df = train_test_split(trainval_df, test_size=0.05, random_state=10, stratify=trainval_df['label'])

    train_df = train_df[['text', 'label']]
    val_df = val_df[['text', 'label']]
    test_df = test_df[['text', 'label']]

    #create dataset
    train_dataset = Dataset.from_dict(train_df)
    val_dataset =  Dataset.from_dict(val_df)
    test_dataset = Dataset.from_dict(test_df)

    issue_dataset = datasets.DatasetDict({"train":train_dataset,"val":val_dataset,"test":test_dataset})

    #for class weight 
    count_0 = len(train_df[train_df['label'] == 0])
    count_1 = len(train_df[train_df['label'] == 1])
    count_2 = len(train_df[train_df['label'] == 2])
    count_3 = len(train_df[train_df['label'] == 3])
    count_4 = len(train_df[train_df['label'] == 4])
    count_5 = len(train_df[train_df['label'] == 5])
    count_6 = len(train_df[train_df['label'] == 6])
    count_7 = len(train_df[train_df['label'] == 7])
    count_8 = len(train_df[train_df['label'] == 8])
    count_9 = len(train_df[train_df['label'] == 9])
    count_10 = len(train_df[train_df['label'] == 10])
    count_11 = len(train_df[train_df['label'] == 11])


    class_weight_0 = (1 / count_0) * (len(train_df) / len(set(train_df['label'])))
    class_weight_1 = (1 / count_1) * (len(train_df) / len(set(train_df['label'])))
    class_weight_2 = (1 / count_2) * (len(train_df) / len(set(train_df['label'])))
    class_weight_3 = (1 / count_3) * (len(train_df) / len(set(train_df['label'])))
    class_weight_4 = (1 / count_4) * (len(train_df) / len(set(train_df['label'])))
    class_weight_5 = (1 / count_5) * (len(train_df) / len(set(train_df['label'])))
    class_weight_6 = (1 / count_6) * (len(train_df) / len(set(train_df['label'])))
    class_weight_7 = (1 / count_7) * (len(train_df) / len(set(train_df['label'])))
    class_weight_8 = (1 / count_8) * (len(train_df) / len(set(train_df['label'])))
    class_weight_9 = (1 / count_9) * (len(train_df) / len(set(train_df['label'])))
    class_weight_10 = (1 / count_10) * (len(train_df) / len(set(train_df['label'])))
    class_weight_11 = (1 / count_11) * (len(train_df) / len(set(train_df['label'])))

    tokenized_issue_dataset = issue_dataset.map(preprocess_function, batched=True)



    #trainer
    class CustomTrainer(Trainer):
        def compute_loss(self, model, inputs, return_outputs=False):
            device = model.device
            labels = inputs.get("labels").to(device)
            # forward pass
            outputs = model(**inputs)
            logits = outputs.get("logits").to(device)
            # compute custom loss (suppose one has 3 labels with different weights)
            loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([class_weight_0,class_weight_1,class_weight_2,class_weight_3,class_weight_4,class_weight_5,class_weight_6,class_weight_7,class_weight_8,class_weight_9,class_weight_10,class_weight_11])).to(device)
            loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
            return (loss, outputs) if return_outputs else loss




    #Training Arguments
    training_args = TrainingArguments(
        output_dir="./results",
        learning_rate=2e-5,
        per_device_train_batch_size=16, #16
        per_device_eval_batch_size=16, #16
        num_train_epochs=10,
        weight_decay=0.01,
    )

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_issue_dataset["train"],
        eval_dataset=tokenized_issue_dataset["val"],
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    #Training
    trainer.train()


    #Testing
    
    # Use the model to get predictions
    test_predictions = trainer.predict(tokenized_issue_dataset["test"])
    # For each prediction, create the label with argmax
    test_predictions_argmax = np.argmax(test_predictions[0], axis=1)

    print(classification_report(np.array(test_df['label'].to_list()), test_predictions_argmax))
    report= classification_report(np.array(test_df['label'].to_list()), test_predictions_argmax, output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    report_df.to_csv('fold'+str(fold)+'.csv', index = False)

#Training and Testing of LightGBM
We have used cross fold validation where the dataset has been split into 10-folds using a stratified sampling. Each fold contains the data of each class in the same proportion and no two folds has any overlap between themselves

In [ ]:
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn import preprocessing
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import classification_report
NUMBER_OF_CLASS=12

issue_df = pd.read_csv(datapath)

label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(issue_df['Code'])
issue_df['Code (Original)'] = issue_df['Code']
issue_df['Code'] = label_encoder.transform(issue_df['Code'])

#number_of_class
class_id_map_df = issue_df.drop_duplicates(subset=['Code'])
total_class=len(class_id_map_df)
encoded_label_list = class_id_map_df['Code'].to_list()
original_label_list = class_id_map_df['Code (Original)'].to_list()
class_id_map = dict(zip(encoded_label_list, original_label_list))

categ = ['aa','begauth','has_code','first_turn','last_turn']
le = preprocessing.LabelEncoder()
issue_df[categ] = issue_df[categ].apply(le.fit_transform)

#cross-fold validation
skf = StratifiedKFold(n_splits=10, random_state=10, shuffle=True)

fold=0
for train_index, test_index in skf.split(issue_df,issue_df['Code']):  
    fold=fold+1
    train_df = issue_df.iloc[train_index]
    test_df = issue_df.iloc[test_index]
    X_train = train_df[['len','tloc','cloc','tpos1','tpos2','clen','tlen','ppau','npau','aa','begauth','has_code','first_turn','last_turn']] #'Full Length',
    y_train = train_df['Code']

    X_test = test_df[['len','tloc','cloc','tpos1','tpos2','clen','tlen','ppau','npau','aa','begauth','has_code','first_turn','last_turn']] #'Full Length',
    y_test = test_df['Code']


    #change the parameters by searching on optuna for different cases 
    clf = lgb.LGBMClassifier(objective="multiclass", class_weight='balanced', num_classes=13, metric='multi_logloss', feature_pre_filter= False,
                             boosting_type="gbdt", lambda_l1=5.6172275504022865e-05, lambda_l2= 0.10617995541784066,feature_fraction= 0.7,
                             bagging_fraction= 1.0, bagging_freq= 0, min_child_samples= 20,num_leaves= 127)
    
    clf.fit(X_train, y_train)
    y_pred=clf.predict(X_test)
    label_list = []


    for i in range(NUMBER_OF_CLASS):
      label_list.append(class_id_map[i])

    true_labels = []
    pred_labels = []


    for e in y_test:
      true_labels.append(class_id_map[e])


    for e in y_pred:
      pred_labels.append(class_id_map[e])  
    
    print(classification_report(true_labels,pred_labels))